# DRMT Data

In [1]:
import pandas as pd
from siuba import *
import numpy as np
from IPython.display import Markdown
from calitp import to_snakecase

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 1000)

In [3]:
from calitp.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/grants/'


/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
df_dist = pd.read_excel(f"{GCS_FILE_PATH}Grant+Distributions_3_17_2022.xlsx") 
df_proj = pd.read_excel(f"{GCS_FILE_PATH}Grant+Projects_3_17_2022.xlsx")

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
df_dist.head()

,Calendar Year,Funding Program,Grant Number,AwardedDate,Approved Date,Lapse Date,Closeout Date,Reopened Date,Distribution Name,Distribution Awarded,Distribution Allocated,Distribution Encumbered,Distribution Unallocated,Distribution Expended,Distribution CloseoutBalance
0,2011,Section 5311,CA-18-X047 | 0012000083,2011-07-14,2011-07-14,2016-09-30,NaT,NaN,Madera 2011,"$184,527.00","$184,527.00","$184,527.00",$0.00,"$175,216.02",$0.00
1,2012,Section 5311,CA-18-X052 | 0012000304,2012-07-19,2012-07-19,2017-09-30,NaT,NaN,Madera 2012,"$46,547.00","$46,547.00","$46,547.00",$0.00,"$45,523.81",$0.00
2,2013,5339 (State),CA-34-0038 | 0016000269,2015-11-30,2016-11-05,2019-06-30,NaT,NaN,Main,"$-152,686.00",$0.00,$0.00,"$-152,686.00",$0.00,$0.00
3,2013,5339 (State),CA-34-0038 | 0016000269,2015-11-30,2016-11-05,2019-06-30,NaT,NaN,Sierra County,"$98,967.00",$0.00,$0.00,"$98,967.00",$0.00,$0.00
4,2013,5339 (State),CA-34-0038 | 0016000269,2015-11-30,2016-11-05,2019-06-30,NaT,NaN,City of Chowchilla,"$138,800.00",$0.00,$0.00,"$138,800.00",$0.00,$0.00


In [6]:
df_proj.head()

,Grant Fiscal Year,Funding Program,Grant Number,Project Year,Organization Name,UPIN,Description,ALI,Contract Number,AllocationAmount,Encumbered Amount,ExpendedAmount,ActiveBalance,ClosedOutBalance,Project Status,Project Closed By,Project Closed Date,Project Closed Time
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,City of Chowchilla,BCG0000228,Operating Assistance,300902,64BO17-00368,53221.0,114511.0,53221.00,0.00,0,Open,NaN,NaN,NaN
1,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0,Open,NaN,NaN,NaN
2,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,20643.0,44265.0,20643.00,0.00,0,Open,NaN,NaN,NaN
3,2012,Section 5311,CA-18-X052 | 0012000304,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,23622.0,44265.0,22868.30,753.70,0,Open,NaN,NaN,NaN
4,2012,Section 5311,CA-18-X052 | 0012000304,2016,Madera County,BCG0000286,Purchase Expansion <30ft Bus,111304,64BC17-00480,22925.0,113319.0,22655.51,269.49,0,Open,NaN,NaN,NaN


In [7]:
df_all = pd.merge(df_dist, df_proj, on='Grant Number', how='outer', indicator=True)

In [8]:
df_all._merge.value_counts()

both          68888
left_only        22
right_only        0
Name: _merge, dtype: int64

In [9]:
(df_all>>select(_['Distribution Name'], _['Organization Name'])).head(20)

,Distribution Name,Organization Name
0,Madera 2011,City of Chowchilla
1,Madera 2011,Madera County
2,Madera 2011,Madera County
3,Madera 2012,Madera County
4,Madera 2012,Madera County
5,Main,NaN
6,Sierra County,NaN
7,City of Chowchilla,NaN
8,City of Dinuba,NaN
9,City of Ridgecrest,NaN


In [10]:
df_all["Organization Name"] = df_all["Organization Name"].fillna(df_all["Distribution Name"])

In [11]:
df_all= to_snakecase(df_all)

In [12]:
df_all

,calendar_year,funding_program_x,grant_number,awardeddate,approved_date,lapse_date,closeout_date,reopened_date,distribution_name,distribution_awarded,distribution_allocated,distribution_encumbered,distribution_unallocated,distribution_expended,distribution_closeoutbalance,grant_fiscal_year,funding_program_y,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time,_merge
0,2011,Section 5311,CA-18-X047 | 0012000083,2011-07-14,2011-07-14,2016-09-30,NaT,NaN,Madera 2011,"$184,527.00","$184,527.00","$184,527.00",$0.00,"$175,216.02",$0.00,2011.0,Section 5311,2016.0,City of Chowchilla,BCG0000228,Operating Assistance,300902,64BO17-00368,53221.0,114511.0,53221.00,0.00,0.0,Open,NaN,NaN,NaN,both
1,2011,Section 5311,CA-18-X047 | 0012000083,2011-07-14,2011-07-14,2016-09-30,NaT,NaN,Madera 2011,"$184,527.00","$184,527.00","$184,527.00",$0.00,"$175,216.02",$0.00,2011.0,Section 5311,2016.0,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0.0,Open,NaN,NaN,NaN,both
2,2011,Section 5311,CA-18-X047 | 0012000083,2011-07-14,2011-07-14,2016-09-30,NaT,NaN,Madera 2011,"$184,527.00","$184,527.00","$184,527.00",$0.00,"$175,216.02",$0.00,2011.0,Section 5311,2016.0,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,20643.0,44265.0,20643.00,0.00,0.0,Open,NaN,NaN,NaN,both
3,2012,Section 5311,CA-18-X052 | 0012000304,2012-07-19,2012-07-19,2017-09-30,NaT,NaN,Madera 2012,"$46,547.00","$46,547.00","$46,547.00",$0.00,"$45,523.81",$0.00,2012.0,Section 5311,2016.0,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,23622.0,44265.0,22868.30,753.70,0.0,Open,NaN,NaN,NaN,both
4,2012,Section 5311,CA-18-X052 | 0012000304,2012-07-19,2012-07-19,2017-09-30,NaT,NaN,Madera 2012,"$46,547.00","$46,547.00","$46,547.00",$0.00,"$45,523.81",$0.00,2012.0,Section 5311,2016.0,Madera County,BCG0000286,Purchase Expansion <30ft Bus,111304,64BC17-00480,22925.0,113319.0,22655.51,269.49,0.0,Open,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68905,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,Victor Valley Community Services Council,BCG0003087,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01592,22512.0,22512.0,18011.97,4500.03,0.0,Open,NaN,NaN,NaN,both
68906,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,Victor Valley Transit Authority,BCG0003088,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01593,22511.0,22511.0,22511.00,0.00,0.0,Open,NaN,NaN,NaN,both
68907,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,"Vivalon, Inc.",BCG0003077,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01557,45354.0,45354.0,0.00,45354.00,0.0,Open,NaN,NaN,NaN,both
68908,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,Wilshire Community Services,BCG0003050,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01594,37674.0,37674.0,0.00,37674.00,0.0,Open,NaN,NaN,NaN,both


In [13]:
df_all.grant_fiscal_year.value_counts()

2020.0    19368
2019.0    16646
2017.0    13521
2021.0     9746
2018.0     4802
2016.0     4700
2014.0       64
2013.0       18
2015.0       18
2011.0        3
2012.0        2
Name: grant_fiscal_year, dtype: int64

In [14]:
df_21 = df_all>>filter(_.grant_fiscal_year==2021)

In [15]:
df_21


,calendar_year,funding_program_x,grant_number,awardeddate,approved_date,lapse_date,closeout_date,reopened_date,distribution_name,distribution_awarded,distribution_allocated,distribution_encumbered,distribution_unallocated,distribution_expended,distribution_closeoutbalance,grant_fiscal_year,funding_program_y,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time,_merge
59160,2021,Section 5311,TBD | 0021000227,NaT,NaT,2026-09-30,NaT,NaN,Main,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,2021.0,Section 5311,2021.0,Alpine County Community Development,BCG0002992,Operating Assistance Sliding Scale,300902,NaN,48000.0,0.0,0.00,48000.00,0.0,Open,NaN,NaN,NaN,both
59161,2021,Section 5311,TBD | 0021000227,NaT,NaT,2026-09-30,NaT,NaN,Main,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,2021.0,Section 5311,2021.0,Amador Transit,BCG0002889,Operating Assistance Sliding Scale,300902,NaN,239425.0,0.0,0.00,239425.00,0.0,Open,NaN,NaN,NaN,both
59162,2021,Section 5311,TBD | 0021000227,NaT,NaT,2026-09-30,NaT,NaN,Main,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,2021.0,Section 5311,2021.0,Butte County Association of Governments/ Butte Regional Transit,BCG0002910,Operating Assistance Sliding Scale,300902,NaN,765737.0,0.0,0.00,765737.00,0.0,Open,NaN,NaN,NaN,both
59163,2021,Section 5311,TBD | 0021000227,NaT,NaT,2026-09-30,NaT,NaN,Main,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,2021.0,Section 5311,2021.0,Calaveras Transit Agency,BCG0002797,Operating Assistance Sliding Scale,300902,NaN,286485.0,0.0,0.00,286485.00,0.0,Open,NaN,NaN,NaN,both
59164,2021,Section 5311,TBD | 0021000227,NaT,NaT,2026-09-30,NaT,NaN,Main,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,2021.0,Section 5311,2021.0,City of Arvin,BCG0002995,Operating Assistance Sliding Scale,300902,NaN,76034.0,0.0,0.00,76034.00,0.0,Open,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68904,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,Valley Achievement Center,BCG0003024,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01591,34172.0,34172.0,0.00,34172.00,0.0,Open,NaN,NaN,NaN,both
68905,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,Victor Valley Community Services Council,BCG0003087,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01592,22512.0,22512.0,18011.97,4500.03,0.0,Open,NaN,NaN,NaN,both
68906,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,Victor Valley Transit Authority,BCG0003088,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01593,22511.0,22511.0,22511.00,0.00,0.0,Open,NaN,NaN,NaN,both
68907,2021,CRRSAA,CA-2021-136| 0021000251,2021-07-21,2021-07-21,2025-09-30,NaT,NaN,LUZA Visalia,"$32,494.00","$32,494.00","$32,494.00",$0.00,$0.00,$0.00,2021.0,CRRSAA,2021.0,"Vivalon, Inc.",BCG0003077,CRRSAA FTA 5310 - Emergency Relief- Operating Assistance 100% Federal Share,300908,64SO21-01557,45354.0,45354.0,0.00,45354.00,0.0,Open,NaN,NaN,NaN,both


In [26]:
#df_all.to_parquet("drmt_grants.parquet")

* based on inital view, df_dist will provide better insight to the application date

### `df_dist` analysis

In [13]:
df_dist = to_snakecase(df_dist)

In [24]:
df_dist.sample()

,calendar_year,funding_program,grant_number,awardeddate,approved_date,lapse_date,closeout_date,reopened_date,distribution_name,distribution_awarded,distribution_allocated,distribution_encumbered,distribution_unallocated,distribution_expended,distribution_closeoutbalance
159,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017-09-18,2017-10-01,2022-09-30,NaT,NaN,Indio-Cathedral City FFY15-17,"$553,433.00","$553,433.00","$553,433.00",$0.00,"$537,888.08",$0.00


In [17]:
df_dist>>count(_.awardeddate)>>arrange(-_.awardeddate)

,awardeddate,n
27,2022-02-25,66
26,2021-08-04,2
25,2021-07-30,1
24,2021-07-21,16
23,2021-04-16,4
22,2021-03-23,96
21,2020-09-22,48
20,2020-09-11,2
19,2020-09-08,2
18,2020-08-28,34


In [21]:
df_dist>>filter(_.awardeddate=='2020-04-21')>>count(_.distribution_name)>>filter(_.n)

,distribution_name,n
0,Alpine County Community Deevelopment,1
1,Amador Transit,1
2,BCAG / Butte Regional Transit,1
3,Butte CAG,1
4,Calaveras Transit Agency,1
...,...,...
81,Tulomne County Transit Agency (TCTA),1
82,Victor Valley Transit Authority,1
83,Yolo County Transportation District,1
84,Yosemite Area Regional Transit System,1


In [25]:
df_dist.sample()

,calendar_year,funding_program,grant_number,awardeddate,approved_date,lapse_date,closeout_date,reopened_date,distribution_name,distribution_awarded,distribution_allocated,distribution_encumbered,distribution_unallocated,distribution_expended,distribution_closeoutbalance
25,2015,Section 5311,CA-18-X077 | 0016000239,2015-11-30,2015-11-06,2020-09-30,NaT,NaN,Main 2015,$0.00,"$34,569.00","$34,569.00","$-34,569.00","$34,569.00",$0.00


In [26]:
df1 = df_dist >> select(
    _.calendar_year, _.awardeddate, _.approved_date, _.distribution_name
)

In [29]:
df1["award_m"] = pd.DatetimeIndex(df1["awardeddate"]).month
df1["award_y"] = pd.DatetimeIndex(df1["awardeddate"]).year

In [33]:
df1

,calendar_year,awardeddate,approved_date,distribution_name,award_m,award_y
0,2011,2011-07-14,2011-07-14,Madera 2011,7.0,2011.0
1,2012,2012-07-19,2012-07-19,Madera 2012,7.0,2012.0
2,2013,2015-11-30,2016-11-05,Main,11.0,2015.0
3,2013,2015-11-30,2016-11-05,Sierra County,11.0,2015.0
4,2013,2015-11-30,2016-11-05,City of Chowchilla,11.0,2015.0
...,...,...,...,...,...,...
735,2021,2022-02-25,2022-02-28,Victorville-Hesperia,2.0,2022.0
736,2021,2022-02-25,2022-02-28,Visalia,2.0,2022.0
737,2021,2022-02-25,2022-02-28,Small Urban,2.0,2022.0
738,2021,2022-02-25,2022-02-28,Rural,2.0,2022.0


In [34]:
#t3['days_between'] = (t3['prepared_date'] - t3['prepared_date']).

In [37]:
df1['cal_award_diff']= df1["award_y"] - df1["calendar_year"]

In [39]:
df1>>count(_.cal_award_diff)

,cal_award_diff,n
0,-1.0,2
1,0.0,311
2,1.0,301
3,2.0,22
4,3.0,2


In [42]:
df1['award_app_diff']= df1["awardeddate"] - df1["approved_date"]
df1

,calendar_year,awardeddate,approved_date,distribution_name,award_m,award_y,cal_award_diff,award_app_diff
0,2011,2011-07-14,2011-07-14,Madera 2011,7.0,2011.0,0.0,0 days
1,2012,2012-07-19,2012-07-19,Madera 2012,7.0,2012.0,0.0,0 days
2,2013,2015-11-30,2016-11-05,Main,11.0,2015.0,2.0,-341 days
3,2013,2015-11-30,2016-11-05,Sierra County,11.0,2015.0,2.0,-341 days
4,2013,2015-11-30,2016-11-05,City of Chowchilla,11.0,2015.0,2.0,-341 days
...,...,...,...,...,...,...,...,...
735,2021,2022-02-25,2022-02-28,Victorville-Hesperia,2.0,2022.0,1.0,-3 days
736,2021,2022-02-25,2022-02-28,Visalia,2.0,2022.0,1.0,-3 days
737,2021,2022-02-25,2022-02-28,Small Urban,2.0,2022.0,1.0,-3 days
738,2021,2022-02-25,2022-02-28,Rural,2.0,2022.0,1.0,-3 days


In [46]:
df1.award_app_diff.describe()

count                            638
mean      77 days 22:11:39.686520376
std      232 days 13:56:34.421090092
min              -341 days +00:00:00
25%                -3 days +00:00:00
50%                  0 days 00:00:00
75%                  0 days 00:00:00
max                809 days 00:00:00
Name: award_app_diff, dtype: object

In [48]:
(df1 >> group_by(_.distribution_name) >> summarize(n=_.award_app_diff.mean()) >> arrange(-_.n)).head(20)

,distribution_name,n
11,Alpine 2016,809 days 00:00:00
19,Amador 2016,809 days 00:00:00
36,Butte 2016,809 days 00:00:00
44,Calaveras 2016,809 days 00:00:00
75,Colusa 2016,809 days 00:00:00
97,Del Norte 2016,809 days 00:00:00
105,El Dorado 2016,809 days 00:00:00
115,Fresno 2016,809 days 00:00:00
124,Glenn 2016,809 days 00:00:00
136,Humboldt 2016,809 days 00:00:00


In [52]:
df1>>filter(_.award_app_diff=='809 days 00:00:00')>>count(_.distribution_name)>>arrange(-_.n)

,distribution_name,n
0,Alpine 2016,1
1,Amador 2016,1
2,Butte 2016,1
3,Calaveras 2016,1
4,Colusa 2016,1
5,Del Norte 2016,1
6,El Dorado 2016,1
7,Fresno 2016,1
8,Glenn 2016,1
9,Humboldt 2016,1


In [40]:
df1 >> group_by(_.distribution_name) >> summarize(n=_.award_y.nunique()) >> arrange(-_.n)

,distribution_name,n
240,Main,7
14,Alpine 2019,2
22,Amador 2019,2
39,Butte 2019,2
47,Calaveras 2019,2
...,...,...
461,Small Urban 20&21,0
474,Stockton 20&21,0
524,Victorville 20&21,0
529,Visalia 20&21,0


In [41]:
df1 >> group_by(_.distribution_name) >> summarize(n=_.calendar_year.nunique()) >> arrange(-_.n)

,distribution_name,n
240,Main,8
14,Alpine 2019,2
22,Amador 2019,2
39,Butte 2019,2
47,Calaveras 2019,2
...,...,...
531,Visalia FFY15-17,1
532,YARTS,1
533,Yolo County Transportation District,1
534,Yosemite Area Regional Transit System,1
